# Output For ES Group

In [1]:
import json

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from sklearn.preprocessing import normalize

from constants import get_all_emotions

## Data Reading

Example of Input from CV Group, what we expected from them with 25 landmarks.

In [2]:
#This will be replaced with the reading data from CV Group
# Sample data, do not skip
data_df = pd.read_csv(r'../../data/lm3.csv')
data_df = data_df.dropna(axis=1, how='any')
features = [x != 'Label' for x in data_df.columns.values]
all_values = data_df.loc[:, features].values
r = all_values[0]

In [3]:
r

array([-72.961,  -1.725,  22.958, -55.678,   4.591,  38.791, -31.92 ,
        -1.929,  36.645,  -1.467,  -1.246,  39.396,  19.511,   4.733,
        39.521,  38.267,   0.895,  23.847, -62.234, -13.599,  23.387,
       -30.594, -13.468,  27.897,  -1.368, -12.608,  28.601,  30.963,
        -9.038,  19.359, -20.506, -20.528,  43.981,  -6.559, -20.715,
        40.831, -30.866, -52.014,  39.217, -12.796, -44.354,  60.519,
         4.687, -50.481,  37.689, -28.704, -81.568,  31.654, -11.424,
       -77.828,  44.218,   7.849, -78.691,  29.98 ,  -9.891, -86.283,
        39.421])

# Load PCA Model
We are loading the PCA model here which we used while training our classifier in order to apply the same dimensionality reduction to the given input.

In [4]:
# Normalize
all_values = normalize([r], axis=1)
print(all_values)

# Load PCA model and transform input
pca = joblib.load('pca-model.pkl')
principal_components = pca.transform(all_values)
#print(principal_components)

[[-0.25785733 -0.00609646  0.08113771 -0.19677609  0.01622542  0.13709439
  -0.11281103 -0.00681743  0.12951004 -0.00518464 -0.00440359  0.13923257
   0.06895539  0.01672728  0.13967434  0.13524248  0.00316309  0.0842796
  -0.21994618 -0.04806132  0.08265387 -0.10812471 -0.04759834  0.09859303
  -0.00483476 -0.04455894  0.10108109  0.10942882 -0.03194192  0.0684182
  -0.0724719  -0.07254965  0.15543678 -0.02318069 -0.07321054  0.14430411
  -0.10908601 -0.18382685  0.13859995 -0.04522337 -0.15675503  0.21388506
   0.0165647  -0.17840895  0.13319972 -0.10144511 -0.28827602  0.11187094
  -0.04037448 -0.27505818  0.15627438  0.02773978 -0.27810818  0.10595472
  -0.03495658 -0.30493968  0.13932092]]


# SVM Model
Using SVM model trained on bosphorus dataset.

In [5]:
svm = joblib.load('svm-model.pkl')
predicted_labels = svm.predict_proba(principal_components)[0]

# Output
We are sending 'output.json' file to Emotion Synthesis group which has the below printed format. The file contains the label and the percentage by which it predicts the label.

In [6]:
# Assign each score to its label
confidence = {}
labels = get_all_emotions()
for index,emotion in enumerate(labels):
    confidence[emotion] = predicted_labels[index]

json_string = json.dumps(confidence,indent=3)
with open('output.json', 'w') as outfile:
    json.dump(confidence, outfile,indent=3)

print(json_string)

{
   "ANGER": 0.5621434189237174,
   "DISGUST": 0.16585436907814466,
   "FEAR": 0.058895752494619404,
   "HAPPY": 0.0014569470031272024,
   "SADNESS": 0.16397763216461944,
   "SURPRISE": 0.04767188033577185
}
